## POS tagging using modified Viterbi

#### Objectives and Plan for Assignment

###### Step 1 : Write Vanilla Viterbi Algorithm 
- Write Vanilla Viterbi Algorithm for POS tagging using Treebank data with Universal tagset
- Train - Validation set should be kept as 95:5 
- Calculate tagging accuracy using Test Sentence file


###### Step 2: Rule Based
- Write a function for improving Vanilla viterbi by either modifying the entire function or calling another function for rule based modification for unkown words
- compare the tagging accuracy on test file

###### Step 3: Probabilistic 
- wtite a method to improve tagging by making state prob = emission prob for unknown words
- compare the tagging accuracy

###### Step 4: Sight 3 improvements in tagging test file 

### Data Preparation

In [1]:
#Importing libraries
import numpy as np
import math
import pandas as pd
import random
import nltk
from sklearn.model_selection import train_test_split
import time
from nltk.tokenize import word_tokenize
from collections import Counter

In [2]:
# reading the Treebank tagged sentences
nltk_data = list(nltk.corpus.treebank.tagged_sents(tagset='universal'))

In [3]:
nltk_data[:5]

[[('Pierre', 'NOUN'),
  ('Vinken', 'NOUN'),
  (',', '.'),
  ('61', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  (',', '.'),
  ('will', 'VERB'),
  ('join', 'VERB'),
  ('the', 'DET'),
  ('board', 'NOUN'),
  ('as', 'ADP'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
  ('director', 'NOUN'),
  ('Nov.', 'NOUN'),
  ('29', 'NUM'),
  ('.', '.')],
 [('Mr.', 'NOUN'),
  ('Vinken', 'NOUN'),
  ('is', 'VERB'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Elsevier', 'NOUN'),
  ('N.V.', 'NOUN'),
  (',', '.'),
  ('the', 'DET'),
  ('Dutch', 'NOUN'),
  ('publishing', 'VERB'),
  ('group', 'NOUN'),
  ('.', '.')],
 [('Rudolph', 'NOUN'),
  ('Agnew', 'NOUN'),
  (',', '.'),
  ('55', 'NUM'),
  ('years', 'NOUN'),
  ('old', 'ADJ'),
  ('and', 'CONJ'),
  ('former', 'ADJ'),
  ('chairman', 'NOUN'),
  ('of', 'ADP'),
  ('Consolidated', 'NOUN'),
  ('Gold', 'NOUN'),
  ('Fields', 'NOUN'),
  ('PLC', 'NOUN'),
  (',', '.'),
  ('was', 'VERB'),
  ('named', 'VERB'),
  ('*-1', 'X'),
  ('a', 'DET'),
  ('nonexecutive', 'ADJ'),
 

- we can see that nltk_data is a list of sentences tagged with POS tags

In [4]:
# Dividing nltk_data into train and validation set in ratio of 95:5
random.seed(1234)
train_set, valid_set = train_test_split(nltk_data,test_size=0.05)
print(len(train_set))
print(len(valid_set))

3718
196


In [5]:
# Getting list of tagged words
train_tagged_words = [tup for sent in train_set for tup in sent]
len(train_tagged_words)

95782

In [6]:
train_tagged_words[0:5]

[('In', 'ADP'),
 ('August', 'NOUN'),
 (',', '.'),
 ('Temple', 'NOUN'),
 ('sweetened', 'VERB')]

In [7]:
# total training tokens 
train_tokens = [pair[0] for pair in train_tagged_words]
# total training tags 
train_tags = [pair[1] for pair in train_tagged_words]

In [8]:
# unique words in train_set
V = set(train_tokens)
print(len(V))

# unique tags in train set
T = set(train_tags)
print(len(T))

12067
12


- So, there are 12112 unique words in training set
- and there are 12 different tags 

In [9]:
T

{'.',
 'ADJ',
 'ADP',
 'ADV',
 'CONJ',
 'DET',
 'NOUN',
 'NUM',
 'PRON',
 'PRT',
 'VERB',
 'X'}

In [10]:
# finding most common tags in training set
tag_counts = Counter(train_tags)
tag_counts

Counter({'ADP': 9375,
         'NOUN': 27402,
         '.': 11143,
         'VERB': 12908,
         'DET': 8358,
         'PRT': 3078,
         'NUM': 3346,
         'X': 6298,
         'CONJ': 2161,
         'ADJ': 6087,
         'PRON': 2606,
         'ADV': 3020})

In [11]:
tag_counts.most_common()[0]

('NOUN', 27402)

##### Tag	Meaning	English Examples
- ADJ	adjective	new, good, high, special, big, local
- ADP	adposition	on, of, at, with, by, into, under
- ADV	adverb	really, already, still, early, now
- CONJ	conjunction	and, or, but, if, while, although
- DET	determiner, article	the, a, some, most, every, no, which
- NOUN	noun	year, home, costs, time, Africa
- NUM	numeral	twenty-four, fourth, 1991, 14:24
- PRT	particle	at, on, out, over per, that, up, with
- PRON	pronoun	he, their, her, its, my, I, us
- VERB	verb	is, say, told, given, playing, would
- .	punctuation marks	. , ; !
- X	other	ersatz, esprit, dunno, gr8, univeristy

### Build the vanilla Viterbi based POS tagger

In [12]:
# Emission Probability

def word_given_tag(word, tag, train_bag = train_tagged_words):
    tag_list = [pair for pair in train_bag if pair[1]==tag]
    count_tag = len(tag_list)
    w_given_tag_list = [pair[0] for pair in tag_list if pair[0]==word]
    count_w_given_tag = len(w_given_tag_list)
    
    return (count_w_given_tag, count_tag)

In [13]:
# compute tag given tag: tag2(t2) given tag1 (t1), i.e. Transition Probability

def t2_given_t1(t2, t1, train_bag = train_tagged_words):
    tags = [pair[1] for pair in train_bag]
    count_t1 = len([t for t in tags if t==t1])
    count_t2_t1 = 0
    for index in range(len(tags)-1):
        if tags[index]==t1 and tags[index+1] == t2:
            count_t2_t1 += 1
    return (count_t2_t1, count_t1)

In [14]:
# creating t x t transition matrix of tags
# each column is t2, each row is t1
# thus M(i, j) represents P(tj given ti)

tags_matrix = np.zeros((len(T), len(T)), dtype='float32')
for i, t1 in enumerate(list(T)):
    for j, t2 in enumerate(list(T)): 
        tags_matrix[i, j] = t2_given_t1(t2, t1)[0]/t2_given_t1(t2, t1)[1]

In [15]:
tags_matrix

array([[1.70666669e-02, 3.25120002e-01, 3.54133323e-02, 6.98666647e-02,
        8.42666626e-03, 3.20426673e-01, 1.05279997e-01, 3.94666679e-02,
        6.30400032e-02, 1.49333337e-03, 1.36533333e-02, 7.46666687e-04],
       [9.57166776e-03, 5.38406335e-03, 4.53457758e-02, 3.70902126e-03,
        3.90045457e-02, 6.37592733e-01, 2.05671206e-01, 1.77075863e-02,
        2.24934202e-02, 2.39291694e-04, 1.28021054e-02, 4.78583388e-04],
       [1.42902508e-01, 5.50968572e-02, 7.52619877e-02, 5.52556366e-02,
        2.03715459e-01, 6.16068579e-02, 1.76246427e-02, 1.64020330e-01,
        1.90536678e-03, 1.86408386e-01, 2.61987932e-02, 1.00031756e-02],
       [2.30237916e-02, 8.82578641e-03, 8.86415988e-02, 7.67459720e-03,
        4.85418260e-01, 2.13353798e-01, 7.36761317e-02, 4.06753644e-02,
        7.29086716e-03, 1.22793550e-02, 3.41519564e-02, 4.98848828e-03],
       [9.06414613e-02, 1.36117131e-01, 2.17926860e-01, 3.60241719e-02,
        1.68887511e-01, 1.09621942e-01, 6.46885633e-02, 3.47

In [16]:
# convert the matrix to a df for better readability
tags_df = pd.DataFrame(tags_matrix, columns = list(T), index=list(T))

In [17]:
tags_df

,ADP,DET,X,PRON,VERB,NOUN,ADJ,.,NUM,PRT,ADV,CONJ
ADP,0.017067,0.325120,0.035413,0.069867,0.008427,0.320427,0.105280,0.039467,0.063040,0.001493,0.013653,0.000747
DET,0.009572,0.005384,0.045346,0.003709,0.039005,0.637593,0.205671,0.017708,0.022493,0.000239,0.012802,0.000479
X,0.142903,0.055097,0.075262,0.055256,0.203715,0.061607,0.017625,0.164020,0.001905,0.186408,0.026199,0.010003
PRON,0.023024,0.008826,0.088642,0.007675,0.485418,0.213354,0.073676,0.040675,0.007291,0.012279,0.034152,0.004988
VERB,0.090641,0.136117,0.217927,0.036024,0.168888,0.109622,0.064689,0.034785,0.022777,0.031608,0.081655,0.005268
NOUN,0.177578,0.013138,0.029049,0.004452,0.146595,0.263375,0.012225,0.240530,0.009342,0.043938,0.017189,0.042588
ADJ,0.078035,0.005093,0.021028,0.000493,0.012321,0.698538,0.066535,0.064564,0.020864,0.011171,0.004764,0.016593
.,0.090550,0.174729,0.027282,0.065961,0.088666,0.221215,0.044333,0.092973,0.080409,0.002423,0.052051,0.059320
NUM,0.034967,0.003288,0.212791,0.001494,0.018530,0.352361,0.034668,0.117155,0.181112,0.026300,0.002989,0.014345
PRT,0.020143,0.103964,0.013970,0.017869,0.402209,0.245939,0.084146,0.041910,0.055556,0.001949,0.010071,0.002274


In [18]:
# Viterbi Heuristic
def vanilla_Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = [] 
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p * transition_p    
            p.append(state_probability)
            
        pmax = max(p)
        # getting state for which probability is maximum
        state_max = T[p.index(pmax)] 
        state.append(state_max)
    return list(zip(words, state))

In [22]:
"""Evaluating on Validation Set for 5 Sentences in validation set as it was taking 
more than 1 hr to run for entire validation set adn we would have to do it 3 times 
"""
random.seed(1234)

# choose random 10 sents
rndom = [random.randint(1,len(valid_set)) for x in range(10)]

# list of sents
valid_run = [valid_set[i] for i in rndom]

#list of tuples - validation taged words
valid_tagged_words = [tup for sent in valid_run for tup in sent]
print(len(valid_tagged_words))

#list of words in validation set
valid_words = [tup[0] for sent in valid_run for tup in sent]
print(len(valid_words))

264
264


In [20]:
#tagging the validation words using vanilla viterbi

start = time.time()
vanilla_valid_tagged_seq = vanilla_Viterbi(valid_words)
end = time.time()
difference = end-start

In [21]:
print("Time taken in seconds: ", difference)
print(vanilla_valid_tagged_seq)

Time taken in seconds:  62.845826625823975
[('*-1', 'X'), ('To', 'PRT'), ('offset', 'VERB'), ('the', 'DET'), ('reduction', 'NOUN'), (',', '.'), ('Congress', 'NOUN'), ('approved', 'VERB'), ('a', 'DET'), ('$', '.'), ('20,000', 'NUM'), ('*U*', 'X'), ('fee', 'NOUN'), ('that', 'ADP'), ('investors', 'NOUN'), ('and', 'CONJ'), ('companies', 'NOUN'), ('will', 'VERB'), ('have', 'VERB'), ('*-3', 'X'), ('to', 'PRT'), ('pay', 'VERB'), ('*T*-2', 'X'), ('each', 'DET'), ('time', 'NOUN'), ('0', 'X'), ('they', 'PRON'), ('make', 'VERB'), ('required', 'VERB'), ('filings', 'NOUN'), ('to', 'PRT'), ('antitrust', 'ADJ'), ('regulators', 'NOUN'), ('about', 'ADP'), ('mergers', 'NOUN'), (',', '.'), ('acquisitions', 'NOUN'), ('and', 'CONJ'), ('certain', 'ADJ'), ('other', 'ADJ'), ('transactions', 'NOUN'), ('*T*-4', 'X'), ('.', '.'), ('Volume', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('second', 'ADJ'), ('section', 'NOUN'), ('was', 'VERB'), ('estimated', 'VERB'), ('*-1', 'X'), ('at', 'ADP'), ('18', 'NUM'), ('million'

In [23]:
len(vanilla_valid_tagged_seq)

264

In [24]:
#calculating vanilla accuracy on validation set(10 sentence)

vanilla_check = [i for i, j in zip(vanilla_valid_tagged_seq, valid_tagged_words) if i == j]
vanilla_accuracy = len(vanilla_check)/len(vanilla_valid_tagged_seq)

print("Accuracy on validation set using Vanilla Viterbi: ",round(vanilla_accuracy*100,2),"%")

Accuracy on validation set using Vanilla Viterbi:  91.29 %


In [28]:
valid_tagged_words

[('*-1', 'X'),
 ('To', 'PRT'),
 ('offset', 'VERB'),
 ('the', 'DET'),
 ('reduction', 'NOUN'),
 (',', '.'),
 ('Congress', 'NOUN'),
 ('approved', 'VERB'),
 ('a', 'DET'),
 ('$', '.'),
 ('20,000', 'NUM'),
 ('*U*', 'X'),
 ('fee', 'NOUN'),
 ('that', 'ADP'),
 ('investors', 'NOUN'),
 ('and', 'CONJ'),
 ('companies', 'NOUN'),
 ('will', 'VERB'),
 ('have', 'VERB'),
 ('*-3', 'X'),
 ('to', 'PRT'),
 ('pay', 'VERB'),
 ('*T*-2', 'X'),
 ('each', 'DET'),
 ('time', 'NOUN'),
 ('0', 'X'),
 ('they', 'PRON'),
 ('make', 'VERB'),
 ('required', 'VERB'),
 ('filings', 'NOUN'),
 ('to', 'PRT'),
 ('antitrust', 'ADJ'),
 ('regulators', 'NOUN'),
 ('about', 'ADP'),
 ('mergers', 'NOUN'),
 (',', '.'),
 ('acquisitions', 'NOUN'),
 ('and', 'CONJ'),
 ('certain', 'ADJ'),
 ('other', 'ADJ'),
 ('transactions', 'NOUN'),
 ('*T*-4', 'X'),
 ('.', '.'),
 ('Volume', 'NOUN'),
 ('in', 'ADP'),
 ('the', 'DET'),
 ('second', 'ADJ'),
 ('section', 'NOUN'),
 ('was', 'VERB'),
 ('estimated', 'VERB'),
 ('*-1', 'X'),
 ('at', 'ADP'),
 ('18', 'NUM'),
 

In [29]:
vanilla_incorrect_tagged = [[valid_tagged_words[i-1],j] for i, j in enumerate(zip(vanilla_valid_tagged_seq, valid_tagged_words)) if j[0]!=j[1]]

In [30]:
vanilla_incorrect_tagged

[[('less', 'ADJ'), (('parched', 'ADP'), ('parched', 'VERB'))],
 [('had', 'VERB'), (('little', 'ADV'), ('little', 'ADJ'))],
 [('production', 'NOUN'), (('profited', 'ADP'), ('profited', 'VERB'))],
 [('*-1', 'X'), (('Moving', 'ADP'), ('Moving', 'VERB'))],
 [('he', 'PRON'), (('graduated', 'ADP'), ('graduated', 'VERB'))],
 [('graduated', 'VERB'), (('Phi', 'ADP'), ('Phi', 'NOUN'))],
 [('Phi', 'NOUN'), (('Beta', 'ADP'), ('Beta', 'NOUN'))],
 [('Beta', 'NOUN'), (('Kappa', 'ADP'), ('Kappa', 'NOUN'))],
 [('of', 'ADP'), (('Kentucky', 'ADP'), ('Kentucky', 'NOUN'))],
 [('*', 'X'), (('spending', 'NOUN'), ('spending', 'VERB'))],
 [('for', 'ADP'), (('dairy', 'ADP'), ('dairy', 'NOUN'))],
 [('regulated', 'VERB'), (('*-134', 'ADP'), ('*-134', 'X'))],
 [('which', 'DET'), (('*T*-218', 'ADP'), ('*T*-218', 'X'))],
 [('*T*-218', 'X'), (('reports', 'NOUN'), ('reports', 'VERB'))],
 [('Agriculture', 'NOUN'), (('committees', 'ADP'), ('committees', 'NOUN'))],
 [('*', 'X'), (('Determining', 'ADP'), ('Determining', '

In [32]:
# let's see if there are unknown words in validation set 

unknown_valid_words =[]

for word in valid_words:
    if word not in train_tokens:
        #if word not in unknown_valid_words:
        unknown_valid_words.append(word)
    else: pass
    
#l = len(unknown_words)
#u = len(set(test_untagged_words))
print(len(unknown_valid_words))
#print(len(set(valid_words)))
print(len(valid_words))
print(len(unknown_valid_words)/len(valid_words))
unknown_valid_words

16
264
0.06060606060606061


['parched',
 'profited',
 'Moving',
 'graduated',
 'Phi',
 'Beta',
 'Kappa',
 'Kentucky',
 'dairy',
 '*-134',
 '*T*-218',
 'committees',
 'Determining',
 'enable',
 '*T*-167',
 'wins']

#### Evaluating model on Unseen Test Data

In [33]:
test_sent=[]
f = open("sampleTestSentences.txt",'r')
f1 = f.readlines()
for line in f1:
    test_sent.append(line)
f.close
test_sent

['Android is a mobile operating system developed by Google.\n',
 'Android has been the best-selling OS worldwide on smartphones since 2011 and on tablets since 2013.\n',
 "Google and Twitter made a deal in 2015 that gave Google access to Twitter's firehose.\n",
 'Twitter is an online news and social networking service on which users post and interact with messages known as tweets.\n',
 'Before entering politics, Donald Trump was a domineering businessman and a television personality.\n',
 'The 2018 FIFA World Cup is the 21st FIFA World Cup, an international football tournament contested once every four years.\n',
 'This is the first World Cup to be held in Eastern Europe and the 11th time that it has been held in Europe.\n',
 'Show me the cheapest round trips from Dallas to Atlanta\n',
 'I would like to see flights from Denver to Philadelphia.\n',
 'Show me the price of the flights leaving Atlanta at about 3 in the afternoon and arriving in San Francisco.\n',
 'NASA invited social medi

In [34]:
test_untagged_tokens =[]
for sent in test_sent:
    test_untagged_tokens.append(word_tokenize(sent))
    
test_untagged_words=list(word for i in test_untagged_tokens for word in i)

#list of words in validation set
#valid_words = [tup[0] for sent in valid_run for tup in sent]

In [35]:
start = time.time()
test_vanilla_tagged_seq = vanilla_Viterbi(test_untagged_words)
end = time.time()
difference = end-start
print(difference)
test_vanilla_tagged_seq

39.999496936798096


[('Android', 'ADP'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'ADP'),
 ('.', '.'),
 ('Android', 'ADP'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'ADP'),
 ('worldwide', 'ADP'),
 ('on', 'ADP'),
 ('smartphones', 'ADP'),
 ('since', 'ADP'),
 ('2011', 'ADP'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'ADP'),
 ('.', '.'),
 ('Google', 'ADP'),
 ('and', 'CONJ'),
 ('Twitter', 'ADP'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'ADP'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'ADP'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'ADP'),
 ("'s", 'PRT'),
 ('firehose', 'ADP'),
 ('.', '.'),
 ('Twitter', 'ADP'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADP'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('service', 'NOU

In [37]:
# listing words from test set which were not in train set 

unknown_test_words =[]

for word in test_untagged_words:
    if word not in train_tokens:
        if word not in unknown_test_words:
            unknown_test_words.append(word)
    else: pass
    
#l = len(unknown_words)
#u = len(set(test_untagged_words))
print(len(unknown_test_words))
print(len(set(test_untagged_words)))
print(len(unknown_test_words)/len(set(test_untagged_words)))
unknown_test_words

28
115
0.24347826086956523


['Android',
 'Google',
 'OS',
 'worldwide',
 'smartphones',
 '2011',
 '2013',
 'Twitter',
 '2015',
 'firehose',
 'online',
 'interact',
 'messages',
 'tweets',
 'domineering',
 'personality',
 '2018',
 'FIFA',
 'Cup',
 '21st',
 'tournament',
 'contested',
 'trips',
 'arriving',
 'NASA',
 'invited',
 'ICESAT-2',
 'Satellite']

#### Note
- so there are 28 such unknown words in Test sentence file that were not present in train set
- and all unknown words are tagged wrongly as the first tag in tagging list

In [38]:
## Using nltk.pos_tag tagger which is a perceptron tagger trained on nltk wall stree journal data to tag test sentences 

test_pos_tags = nltk.pos_tag(test_untagged_words, tagset='universal')

In [39]:
test_pos_tags

[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'DET'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'ADJ'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('se

In [115]:
#calculating vanilla accuracy on test set(10 sentence)

test_vanilla_check = [i for i, j in zip(test_vanilla_tagged_seq, test_pos_tags) if i == j]
test_vanilla_accuracy = len(test_vanilla_check)/len(test_vanilla_tagged_seq)

print("Accuracy on test set using Vanilla Viterbi: ",round(test_vanilla_accuracy*100,2),"%")

Accuracy on test set using Vanilla Viterbi:  76.24 %


- clearly the loss in accuracy is due to presence of nearly 25% of unique unknown words(29) out of unique words(115) in test set

## Solve the problem of unknown words

### Approach - 1 # Lexicon and Rules Based Tagging of Unknown Words 

In [93]:
# creating a rule based patterns
## we have seen in incorrectly tagged words in validation set that many 'ing' ending words should be verb
## any number should be labelled as number 
## 'ed' ending should be tagged as Adv
## any other scenario should be tagged as Noun

patterns = [
    (r'.*ing$', 'VERB'),              # gerund
    (r'.*ed$', 'VERB'),              # past tense
    (r'[0-9]{1,}', 'NUM'),            # cardinal numbers
    (r'.*', 'NOUN')                    # nouns
]

In [94]:
# rule based tagger
rule_based_tagger = nltk.RegexpTagger(patterns)

In [95]:
# lexicon backed up by the rule-based tagger
lexicon_tagger = nltk.UnigramTagger(train_set, backoff=rule_based_tagger,)

In [96]:
#bigram tagger
bigram_tagger = nltk.BigramTagger(train_set, backoff=lexicon_tagger)

In [97]:
bigram_tagger.evaluate(valid_set)

0.9507560277891296

In [98]:
bigram_tagger.tag(['Android'])

[('Android', 'NOUN')]

In [59]:
def unknown_rules_tagger(word):
    #token = word_tokenize(word)
    state = bigram_tagger.tag(token)
    return state

In [61]:
state = []
state.append(unknown_rules_tagger('Android'))
#abc = unknown_rules_tagger('Android')
print(abc)
print(abc[0][1])

[('Android', 'Noun')]
Noun


In [80]:
words = valid_words
state = []
T = list(set([pair[1] for pair in train_tagged_words]))
#print(T)
for key, word in enumerate(words):
    #print(key, word)
    p=[]
    for tag in T:
        if key == 0:
            transition_p = tags_df.loc['.', tag]
        else:
            transition_p = tags_df.loc[state[-1], tag]      
        emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
        state_probability = emission_p *transition_p
        p.append(state_probability)
    print(p)
"""

['ADP', 'DET', 'X', 'PRON', 'VERB', 'NOUN', 'ADJ', '.', 'NUM', 'PRT', 'ADV', 'CONJ']


"\nfor key, word in enumerate(words):\n    #print(key, word)\n    p=[]\n    for tag in T:\n        if key == 0:\n            transition_p = tags_df.loc['.', tag]\n        else:\n            transition_p = tags_df.loc[state[-1], tag]      \n        emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]\n        state_probability = emission_p *transition_p\n        p.append(state_probability)\n    print(p)\n"

In [90]:
train_tagged_words[0:2]

[('In', 'ADP'), ('August', 'NOUN')]

In [99]:
# Modified Viterbi Model for handling unknown words
def rules_Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        #p1 = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p *transition_p
            p.append(state_probability)
            #if emission_p == 0:
                #state_probability_u = transition_p
                #p1.append(state_probability_u)
                               
        #set_state_probability = set(p)
        #set_state_probability.remove(0.0)
        if sum(p)>0:
            pmax = max(p) # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
            state.append(state_max)
            #print(state)
        else:
            #pmax1 = max(p1)            # getting state for which probability is maximum
            #state_max = T[p1.index(pmax1)] 
            state.append(bigram_tagger.tag([word])[0][1])
            
    return list(zip(words, state))

In [100]:
#tagging the validation words using rules viterbi

start = time.time()
rules_valid_tagged_seq = rules_Viterbi(valid_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(rules_valid_tagged_seq)

Time taken in seconds:  78.38528609275818
[('*-1', 'X'), ('To', 'PRT'), ('offset', 'VERB'), ('the', 'DET'), ('reduction', 'NOUN'), (',', '.'), ('Congress', 'NOUN'), ('approved', 'VERB'), ('a', 'DET'), ('$', '.'), ('20,000', 'NUM'), ('*U*', 'X'), ('fee', 'NOUN'), ('that', 'ADP'), ('investors', 'NOUN'), ('and', 'CONJ'), ('companies', 'NOUN'), ('will', 'VERB'), ('have', 'VERB'), ('*-3', 'X'), ('to', 'PRT'), ('pay', 'VERB'), ('*T*-2', 'X'), ('each', 'DET'), ('time', 'NOUN'), ('0', 'X'), ('they', 'PRON'), ('make', 'VERB'), ('required', 'VERB'), ('filings', 'NOUN'), ('to', 'PRT'), ('antitrust', 'ADJ'), ('regulators', 'NOUN'), ('about', 'ADP'), ('mergers', 'NOUN'), (',', '.'), ('acquisitions', 'NOUN'), ('and', 'CONJ'), ('certain', 'ADJ'), ('other', 'ADJ'), ('transactions', 'NOUN'), ('*T*-4', 'X'), ('.', '.'), ('Volume', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('second', 'ADJ'), ('section', 'NOUN'), ('was', 'VERB'), ('estimated', 'VERB'), ('*-1', 'X'), ('at', 'ADP'), ('18', 'NUM'), ('million',

In [101]:
len(rules_valid_tagged_seq)

264

In [102]:
#calculating rules accuracy on validation set(10 sentence)

rules_check = [i for i, j in zip(rules_valid_tagged_seq, valid_tagged_words) if i == j]
rules_accuracy = len(rules_check)/len(rules_valid_tagged_seq)

print("Accuracy on validation set using prob Viterbi: ",round(rules_accuracy*100,2),"%")

Accuracy on validation set using prob Viterbi:  95.08 %


In [103]:
start = time.time()
test_rules_tagged_seq = rules_Viterbi(test_untagged_words)
end = time.time()
difference = end-start
print(difference)
test_rules_tagged_seq

44.66537666320801


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'NOUN'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'NOUN'),
 ('since', 'ADP'),
 ('2011', 'NUM'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'NUM'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'NUM'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'NOUN'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'NOUN'),
 ("'s", 'PRT'),
 ('firehose', 'NOUN'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('s

In [104]:
#calculating vanilla accuracy on validation set(10 sentence)

test_rules_check = [i for i, j in zip(test_rules_tagged_seq, test_pos_tags) if i == j]
test_rules_accuracy = len(test_rules_check)/len(test_rules_tagged_seq)

print("Accuracy on validation set using prob Viterbi: ",round(test_rules_accuracy*100,2),"%")

Accuracy on validation set using prob Viterbi:  94.48 %


## Approach - 2 # Probabilistic Approach

In [51]:
# Modified Viterbi Model for handling unknown words
def prob2_Viterbi(words, train_bag = train_tagged_words):
    state = []
    T = list(set([pair[1] for pair in train_bag]))
    
    for key, word in enumerate(words):
        #initialise list of probability column for a given observation
        p = []
        p1 = []
        for tag in T:
            if key == 0:
                transition_p = tags_df.loc['.', tag]
            else:
                transition_p = tags_df.loc[state[-1], tag]
                
            # compute emission and state probabilities
            emission_p = word_given_tag(words[key], tag)[0]/word_given_tag(words[key], tag)[1]
            state_probability = emission_p *transition_p
            p.append(state_probability)
            if emission_p == 0:
                state_probability_u = transition_p
                p1.append(state_probability_u)
                               
        #set_state_probability = set(p)
        #set_state_probability.remove(0.0)
        if sum(p)>0:
            pmax = max(p) # getting state for which probability is maximum
            state_max = T[p.index(pmax)]
            state.append(state_max)
        else:
            pmax1 = max(p1)            # getting state for which probability is maximum
            state_max = T[p1.index(pmax1)] 
            state.append(state_max)
            
    return list(zip(words, state))

In [108]:
#tagging the validation words using prob2 viterbi

start = time.time()
prob2_valid_tagged_seq = prob2_Viterbi(valid_words)
end = time.time()
difference = end-start
print("Time taken in seconds: ", difference)
print(prob2_valid_tagged_seq)

Time taken in seconds:  69.84058880805969
[('*-1', 'X'), ('To', 'PRT'), ('offset', 'VERB'), ('the', 'DET'), ('reduction', 'NOUN'), (',', '.'), ('Congress', 'NOUN'), ('approved', 'VERB'), ('a', 'DET'), ('$', '.'), ('20,000', 'NUM'), ('*U*', 'X'), ('fee', 'NOUN'), ('that', 'ADP'), ('investors', 'NOUN'), ('and', 'CONJ'), ('companies', 'NOUN'), ('will', 'VERB'), ('have', 'VERB'), ('*-3', 'X'), ('to', 'PRT'), ('pay', 'VERB'), ('*T*-2', 'X'), ('each', 'DET'), ('time', 'NOUN'), ('0', 'X'), ('they', 'PRON'), ('make', 'VERB'), ('required', 'VERB'), ('filings', 'NOUN'), ('to', 'PRT'), ('antitrust', 'ADJ'), ('regulators', 'NOUN'), ('about', 'ADP'), ('mergers', 'NOUN'), (',', '.'), ('acquisitions', 'NOUN'), ('and', 'CONJ'), ('certain', 'ADJ'), ('other', 'ADJ'), ('transactions', 'NOUN'), ('*T*-4', 'X'), ('.', '.'), ('Volume', 'NOUN'), ('in', 'ADP'), ('the', 'DET'), ('second', 'ADJ'), ('section', 'NOUN'), ('was', 'VERB'), ('estimated', 'VERB'), ('*-1', 'X'), ('at', 'ADP'), ('18', 'NUM'), ('million',

In [109]:
#calculating prob2 accuracy on validation set(10 sentence)

prob2_check = [i for i, j in zip(prob2_valid_tagged_seq, valid_tagged_words) if i == j]
prob2_accuracy = len(prob2_check)/len(prob2_valid_tagged_seq)

print("Accuracy on validation set using prob2 Viterbi: ",round(prob2_accuracy*100,2),"%")

Accuracy on validation set using prob2 Viterbi:  92.8 %


In [52]:
start = time.time()
test_prob2_tagged_seq = prob2_Viterbi(test_untagged_words)
end = time.time()
difference = end-start
print(difference)
test_prob2_tagged_seq

38.11868357658386


[('Android', 'NOUN'),
 ('is', 'VERB'),
 ('a', 'DET'),
 ('mobile', 'ADJ'),
 ('operating', 'NOUN'),
 ('system', 'NOUN'),
 ('developed', 'VERB'),
 ('by', 'ADP'),
 ('Google', 'DET'),
 ('.', '.'),
 ('Android', 'NOUN'),
 ('has', 'VERB'),
 ('been', 'VERB'),
 ('the', 'DET'),
 ('best-selling', 'ADJ'),
 ('OS', 'NOUN'),
 ('worldwide', 'NOUN'),
 ('on', 'ADP'),
 ('smartphones', 'DET'),
 ('since', 'ADP'),
 ('2011', 'DET'),
 ('and', 'CONJ'),
 ('on', 'ADP'),
 ('tablets', 'NOUN'),
 ('since', 'ADP'),
 ('2013', 'DET'),
 ('.', '.'),
 ('Google', 'NOUN'),
 ('and', 'CONJ'),
 ('Twitter', 'NOUN'),
 ('made', 'VERB'),
 ('a', 'DET'),
 ('deal', 'NOUN'),
 ('in', 'ADP'),
 ('2015', 'DET'),
 ('that', 'ADP'),
 ('gave', 'VERB'),
 ('Google', 'X'),
 ('access', 'NOUN'),
 ('to', 'PRT'),
 ('Twitter', 'VERB'),
 ("'s", 'PRT'),
 ('firehose', 'VERB'),
 ('.', '.'),
 ('Twitter', 'NOUN'),
 ('is', 'VERB'),
 ('an', 'DET'),
 ('online', 'NOUN'),
 ('news', 'NOUN'),
 ('and', 'CONJ'),
 ('social', 'ADJ'),
 ('networking', 'NOUN'),
 ('servic

In [53]:
#calculating vanilla accuracy on validation set(10 sentence)

test_prob2_check = [i for i, j in zip(test_prob2_tagged_seq, test_pos_tags) if i == j]
test_prob2_accuracy = len(test_prob2_check)/len(test_prob2_tagged_seq)

print("Accuracy on validation set using prob2 Viterbi: ",round(test_prob2_accuracy*100,2),"%")

Accuracy on validation set using prob2 Viterbi:  86.19 %


##### Evaluating Tagging Accuracy

In [111]:
# Tagging Accuracy of Vanilla Viterbi on Validation Set
print("Tagging Accuracy of Vanilla Viterbi on Validation Set: ",round(vanilla_accuracy*100,2),"%")

Tagging Accuracy of Vanilla Viterbi on Validation Set:  91.29 %


In [112]:
# Tagging Accuracy of Viterbi Modification with Lexicon & Rules based approach on Validation Set
print("Tagging Accuracy of Viterbi with Lexicon & Rules based on Validation Set: ",round(rules_accuracy*100,2),"%")

Tagging Accuracy of Viterbi with Lexicon & Rules based on Validation Set:  95.08 %


In [113]:
# Tagging Accuracy of Viterbi Modification with Probabilistic based approach on Validation Set
print("Accuracy on validation set using prob2 Viterbi: ",round(prob2_accuracy*100,2),"%")

Accuracy on validation set using prob2 Viterbi:  92.8 %


### Compare the tagging accuracies of the modifications with the vanilla Viterbi algorithm

In [116]:
# Tagging Accuracy of Vanilla Viterbi on Test Sentence File
print("Accuracy on test set using Vanilla Viterbi: ",round(test_vanilla_accuracy*100,2),"%")

Accuracy on test set using Vanilla Viterbi:  76.24 %


In [117]:
# Tagging Accuracy of Viterbi Modification with Lexicon & Rules based approach on Test Sentence File
print("Accuracy on validation set using prob Viterbi: ",round(test_rules_accuracy*100,2),"%")

Accuracy on validation set using prob Viterbi:  94.48 %


In [118]:
# Tagging Accuracy of Viterbi Modification with Probabilistic based approach on Validation Set
print("Accuracy on validation set using prob2 Viterbi: ",round(test_prob2_accuracy*100,2),"%")

Accuracy on validation set using prob2 Viterbi:  86.19 %


#### Comments

- As we can clearly see that by applying modifications on Vanilla Viterbi 
- Maximum tagging accuracy was achieved with a Lexicon and Rules based approach for treating unknown words in vanilla viterbi

### List down cases which were incorrectly tagged by original POS tagger and got corrected by your modifications

In [140]:
#Since we are not provided with Pos tags on Test Sentences, we use a nltk.pos_tag which is a perceptron tagger pretrained model on wallstreet journas corpora
# let's see the POS tags on words in Test sentence file which were not present in Train Set or treebank 

actual_tag_unknown_word = [pair for word in unknown_test_words for pair in test_pos_tags if word == pair[0]]
list(set(actual_tag_unknown_word))

[('domineering', 'ADJ'),
 ('worldwide', 'NOUN'),
 ('Cup', 'NOUN'),
 ('2013', 'NUM'),
 ('smartphones', 'NOUN'),
 ('2018', 'NUM'),
 ('trips', 'NOUN'),
 ('tweets', 'NOUN'),
 ('arriving', 'NOUN'),
 ('invited', 'VERB'),
 ('interact', 'NOUN'),
 ('firehose', 'NOUN'),
 ('online', 'ADJ'),
 ('FIFA', 'NOUN'),
 ('tournament', 'NOUN'),
 ('NASA', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('personality', 'NOUN'),
 ('ICESAT-2', 'NOUN'),
 ('21st', 'NUM'),
 ('Android', 'NOUN'),
 ('OS', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('contested', 'VERB'),
 ('messages', 'NOUN'),
 ('2011', 'NUM'),
 ('2015', 'NUM'),
 ('Google', 'NOUN')]

In [143]:
# let's see the POS tags generated by Vanilla Viterbi on Uknown words from test Sentence file (not present in treebank train set) 

vanilla_tag_unknown_word = [pair for word in unknown_test_words for pair in test_vanilla_tagged_seq if word == pair[0]]
list(set(vanilla_tag_unknown_word))

[('Twitter', 'ADP'),
 ('interact', 'ADP'),
 ('2015', 'ADP'),
 ('trips', 'ADP'),
 ('online', 'ADP'),
 ('smartphones', 'ADP'),
 ('firehose', 'ADP'),
 ('messages', 'ADP'),
 ('contested', 'ADP'),
 ('worldwide', 'ADP'),
 ('arriving', 'ADP'),
 ('domineering', 'ADP'),
 ('NASA', 'ADP'),
 ('tournament', 'ADP'),
 ('Satellite', 'ADP'),
 ('2011', 'ADP'),
 ('2018', 'ADP'),
 ('2013', 'ADP'),
 ('ICESAT-2', 'ADP'),
 ('personality', 'ADP'),
 ('tweets', 'ADP'),
 ('invited', 'ADP'),
 ('OS', 'ADP'),
 ('FIFA', 'ADP'),
 ('Google', 'ADP'),
 ('Android', 'ADP'),
 ('21st', 'ADP'),
 ('Cup', 'ADP')]

In [144]:
# let's see the POS tags generated by Viterbi modified with Lexicon & Rules based approach 
# on Uknown words from test Sentence file (not present in treebank train set) 

rules_tag_unknown_word = [pair for word in unknown_test_words for pair in test_rules_tagged_seq if word == pair[0]]
list(set(rules_tag_unknown_word))

[('worldwide', 'NOUN'),
 ('Cup', 'NOUN'),
 ('2013', 'NUM'),
 ('smartphones', 'NOUN'),
 ('online', 'NOUN'),
 ('2018', 'NUM'),
 ('trips', 'NOUN'),
 ('tweets', 'NOUN'),
 ('invited', 'VERB'),
 ('interact', 'NOUN'),
 ('firehose', 'NOUN'),
 ('FIFA', 'NOUN'),
 ('tournament', 'NOUN'),
 ('NASA', 'NOUN'),
 ('domineering', 'VERB'),
 ('Satellite', 'NOUN'),
 ('personality', 'NOUN'),
 ('arriving', 'VERB'),
 ('ICESAT-2', 'NOUN'),
 ('21st', 'NUM'),
 ('Android', 'NOUN'),
 ('OS', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('contested', 'VERB'),
 ('messages', 'NOUN'),
 ('2011', 'NUM'),
 ('2015', 'NUM'),
 ('Google', 'NOUN')]

In [145]:
# let's see the POS tags generated by Viterbi modified with Probabilistic approach 
# on Uknown words from test Sentence file (not present in treebank train set) 

prob2_tag_unknown_word = [pair for word in unknown_test_words for pair in test_prob2_tagged_seq if word == pair[0]]
list(set(prob2_tag_unknown_word))

[('Google', 'X'),
 ('worldwide', 'NOUN'),
 ('Twitter', 'VERB'),
 ('Cup', 'NOUN'),
 ('2011', 'DET'),
 ('online', 'NOUN'),
 ('2013', 'DET'),
 ('trips', 'NOUN'),
 ('ICESAT-2', 'DET'),
 ('arriving', 'NOUN'),
 ('interact', 'NOUN'),
 ('tweets', 'DET'),
 ('Google', 'DET'),
 ('FIFA', 'NOUN'),
 ('tournament', 'NOUN'),
 ('NASA', 'NOUN'),
 ('Satellite', 'NOUN'),
 ('personality', 'NOUN'),
 ('invited', 'NOUN'),
 ('2015', 'DET'),
 ('contested', 'NOUN'),
 ('firehose', 'VERB'),
 ('2018', 'NOUN'),
 ('Android', 'NOUN'),
 ('OS', 'NOUN'),
 ('domineering', 'NOUN'),
 ('Twitter', 'NOUN'),
 ('21st', 'NOUN'),
 ('smartphones', 'DET'),
 ('Google', 'NOUN'),
 ('messages', 'DET')]

##### Summary

- Clearly unknown words in Train set like 'Twitter', 'Google', 'NASA', 'Android' are all getting wrongly tagged as ADP in Vanilla Viterbi model But same were correctly getting tagged as NOUN by applying modifications

- Also Numbers in vanilla viterbi are tagged as ADP but correctly tagged as NUM in Rules based viterbi although they were incorrectly getting tagged as DET in Probabilistic based approach

- also rules based modification on vanilla viterbi was able to resolve some words ending with 'ing' as VERB however here is the limitation of rules based approach as it doesn't captures the dependencies of words with other words in sentences hence limited in capturing contecxt hidden in words

- Overall Lexicon & Rules based Viterbi model was able to perform best as a POS_Tagger compared to Vanilla Viterbi tagger